In [1]:
import torchvision

In [2]:
model = torchvision.models.detection.maskrcnn_resnet50_fpn()

In [3]:
model

MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=1e-05)
          (relu):

In [1]:
import torchvision
import torch.nn as nn


class MaskRcnnMobilenetV3(nn.Module):
    def __init__(self, num_classes, pretrained=True, pretrained_backbone=True):
        super(MaskRcnnMobilenetV3, self).__init__()
        self.model = self.maskrcnn_mobilenetv3(num_classes, pretrained, pretrained_backbone)

    def maskrcnn_mobilenetv3(self, num_classes, pretrained, pretrained_backbone):
        model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_320_fpn(pretrained=pretrained, pretrained_backbone=pretrained_backbone)
        in_features = model.roi_heads.box_predictor.cls_score.in_features
        model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
        return model

    def state_dict(self):
        return self.model.state_dict()

    def load_state_dict(self, state_dict):
        self.model.load_state_dict(state_dict)

    def forward(self, x, targets=None):
        return self.model(x, targets)

In [6]:
model.rpn.anchor_generator.sizes

((32,), (64,), (128,), (256,), (512,))

In [7]:
fasterrcnn_mobilenetv3 = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_320_fpn()

In [14]:
backbone = fasterrcnn_mobilenetv3.backbone
anchor_genterator = fasterrcnn_mobilenetv3.rpn.anchor_generator
rpn_head = fasterrcnn_mobilenetv3.rpn.head


In [15]:
model.backbone = backbone
model.rpn.anchor_generator = anchor_genterator
model.rpn.head = rpn_head

In [16]:
model

MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (0): ConvBNActivation(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): FrozenBatchNorm2d(16, eps=1e-05)
        (2): Hardswish()
      )
      (1): InvertedResidual(
        (block): Sequential(
          (0): ConvBNActivation(
            (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
            (1): FrozenBatchNorm2d(16, eps=1e-05)
            (2): ReLU(inplace=True)
          )
          (1): ConvBNActivation(
            (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): FrozenBatchNorm2d(16, eps=1e-05)
            (2): Identity()
          )
        )
      )
      (2): InvertedResidual(
  

In [19]:
import time
import torch

x = torch.FloatTensor(size=(1, 3, 800, 800))
model.eval()
t1 = time.time()
a = model(x)
t2 = time.time()
print(round(t2 - t1, 5))

0.8239
